<a href="https://colab.research.google.com/github/mzohaibnasir/GenAI/blob/main/09_EndtoEndMedicalChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# End-to-End Medical Chatbot

* will be based on out custom data. (PDF files)corpus would be a book





## Process:

### Backend Component
*** [ Data ingestion ] ***  --->  *** [ extract data/content ] ***  --->  *** [ split into text chunks(multiple chunks to facilitate input context window. context window token limit: 4096 )] ***  --->  *** [embeddings of chunks] ***  --->  *** [build semantic index based on all embeddings (chunk overlap would be beneficial here. Due to this chunk overlap, model would be able to create a relation between different chunks) ] ***  --->  *** [ building Knowledge base using pinecone vector store ]






-------------------------------------------------------------- ---
**RAG is used to fix hellucinationsiu. RAG is base don our own populated vector database.it also gives context based on metadata we input through input docs.**
**In vector db indexes are being updated instantaneously**
**In vector db, neighbors are semantically close**

In vector db, we have two concepts:

### knowledge base
### semantic index
will build clusters based on semantic index(king and queen will be in single cluster based on distance between vectors


-------------------------------------------------
### chunk overlap : is to get some context from previous text.
Chunk overlap is used in LLM (Large Language Models) to address the issue of context windows. LLMs process input text in fixed-size chunks, but sometimes, the context required to understand the text exceeds the chunk size. By using chunk overlap, the model can consider the surrounding context, even if it's in a different chunk.
Here's how it works:
The input text is divided into overlapping chunks, typically with a overlap of 50-100 tokens.
Each chunk is processed separately, but the model has access to the context from the previous chunk (due to the overlap).
This allows the model to capture long-range dependencies and relationships between different parts of the input text.
Chunk overlap helps LLMs to:
Improve context awareness
Enhance understanding of complex texts
Reduce the impact of chunk boundaries on model performance
It's a technique to help LLMs better understand the input text and generate more accurate and coherent responses.